# Microsoft data

In [10]:
from dianet import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    input_data = []
    target_data = []
    
    for i, line in enumerate(lines):
        # print(i, line)
        if i % 2 == 0:  # Input data (rows 1, 3, 5, 7, ...)
            input_data.append([float(x) for x in line.strip().split()])
        else:          # Target data (rows 2, 4, 6, 8, ...)
            target_data.append([float(x) for x in line.strip().split()])

    # Convert to NumPy arrays and reshape
    input_array = np.array(input_data).reshape(-1, 18)
    target_array = np.array(target_data).reshape(-1, 2)
    
    return input_array, target_array

train_in, train_target = read_data('jmeintdata/aggregated_train.txt')
train_target = np.argmax(train_target, axis=1)
print("input:", train_in.shape, "target:", train_target.shape)
train_in, train_target = torch.from_numpy(train_in).float(), torch.from_numpy(train_target).long()

test_in, test_target = read_data('jmeintdata/aggregated_test.txt')
test_target = np.argmax(test_target, axis=1)
print("input:", test_in.shape, "target:", test_target.shape)
test_in, test_target = torch.from_numpy(test_in).float(), torch.from_numpy(test_target).long()

print(train_in[0], train_target[0])

input: (1000, 18) target: (1000,)
input: (1000, 18) target: (1000,)
tensor([0.3729, 0.2780, 0.6589, 0.6045, 0.7331, 0.2824, 0.7268, 0.3357, 0.5670,
        0.6465, 0.5714, 0.7991, 0.4459, 0.4848, 0.9892, 0.0454, 0.1613, 0.3421]) tensor(1)


In [11]:
class dianet20(nn.Module):
    def __init__(self):
        super(dianet20, self).__init__()
        self.fc1, self.fc2, self.fc3, self.fc4 = nn.Linear(18,17), nn.Linear(17,16), nn.Linear(16,15), nn.Linear(15,14)
        self.fc5, self.fc6, self.fc7, self.fc8 = nn.Linear(14,13), nn.Linear(13,12), nn.Linear(12,11), nn.Linear(11,10)
        self.fc9, self.fc10, self.fc11, self.fc12 = nn.Linear(10,9), nn.Linear(9,8), nn.Linear(8,7), nn.Linear(7,6)
        self.fc13, self.fc14, self.fc15, self.fc16 = nn.Linear(6,5), nn.Linear(5,4), nn.Linear(4,3), nn.Linear(3,2)

        self.msk1, self.msk2, self.msk3, self.msk4 = make_mask(17,18).T, make_mask(16,17).T, make_mask(15,16).T, make_mask(14,15).T
        self.msk5, self.msk6, self.msk7, self.msk8 = make_mask(13,14).T, make_mask(12,13).T, make_mask(11,12).T, make_mask(10,11).T
        self.msk9, self.msk10, self.msk11, self.msk12 = make_mask(9,10).T, make_mask(8,9).T, make_mask(7,8).T, make_mask(6,7).T
        self.msk13, self.msk14, self.msk15, self.msk16 = make_mask(5,6).T, make_mask(4,5).T, make_mask(3,4).T, make_mask(2,3).T

    def forward(self, x, dwn, up):
        self.fc1.weight.data *= torch.from_numpy(self.msk1).float()
        self.fc1.weight.data = torch.clamp(self.fc1.weight.data, min=dwn, max=up)
        self.fc1.bias.data = torch.clamp(self.fc1.bias.data, min=dwn, max=up)
        x1 = F.leaky_relu(self.fc1(x), negative_slope=0.125) # x1: 17

        self.fc2.weight.data *= torch.from_numpy(self.msk2).float()
        self.fc2.weight.data = torch.clamp(self.fc2.weight.data, min=dwn, max=up)
        self.fc2.bias.data = torch.clamp(self.fc2.bias.data, min=dwn, max=up)
        x2 = F.leaky_relu((self.fc2(x1)), negative_slope=0.125) # x2: 16

        self.fc3.weight.data *= torch.from_numpy(self.msk3).float()
        self.fc3.weight.data = torch.clamp(self.fc3.weight.data, min=dwn, max=up)
        self.fc3.bias.data = torch.clamp(self.fc3.bias.data, min=dwn, max=up)
        x3 = F.leaky_relu((self.fc3(x2)), negative_slope=0.125) # x3: 15
        x3 = x3+x1[:,1:-1]

        self.fc4.weight.data *= torch.from_numpy(self.msk4).float()
        self.fc4.weight.data = torch.clamp(self.fc4.weight.data, min=dwn, max=up)
        self.fc4.bias.data = torch.clamp(self.fc4.bias.data, min=dwn, max=up)
        x4 = F.leaky_relu((self.fc4(x3)), negative_slope=0.125) # x4: 14
        x4 = x4+x2[:,1:-1]

        self.fc5.weight.data *= torch.from_numpy(self.msk5).float()
        self.fc5.weight.data = torch.clamp(self.fc5.weight.data, min=dwn, max=up)
        self.fc5.bias.data = torch.clamp(self.fc5.bias.data, min=dwn, max=up)
        x5 = F.leaky_relu((self.fc5(x4)), negative_slope=0.125) # x5: 13
        x5 = x5+x3[:,1:-1]

        self.fc6.weight.data *= torch.from_numpy(self.msk6).float()
        self.fc6.weight.data = torch.clamp(self.fc6.weight.data, min=dwn, max=up)
        self.fc6.bias.data = torch.clamp(self.fc6.bias.data, min=dwn, max=up)
        x6 = F.leaky_relu((self.fc6(x5)), negative_slope=0.125) # x6: 12
        x6 = x6+x4[:,1:-1]

        self.fc7.weight.data *= torch.from_numpy(self.msk7).float()
        self.fc7.weight.data = torch.clamp(self.fc7.weight.data, min=dwn, max=up)
        self.fc7.bias.data = torch.clamp(self.fc7.bias.data, min=dwn, max=up)
        x7 = F.leaky_relu((self.fc7(x6)), negative_slope=0.125) # x7: 11
        x7 = x7+x5[:,1:-1]

        self.fc8.weight.data *= torch.from_numpy(self.msk8).float()
        self.fc8.weight.data = torch.clamp(self.fc8.weight.data, min=dwn, max=up)
        self.fc8.bias.data = torch.clamp(self.fc8.bias.data, min=dwn, max=up)
        x8 = F.leaky_relu((self.fc8(x7)), negative_slope=0.125) # x8: 10
        x8 = x8+x6[:,1:-1]

        self.fc9.weight.data *= torch.from_numpy(self.msk9).float()
        self.fc9.weight.data = torch.clamp(self.fc9.weight.data, min=dwn, max=up)
        self.fc9.bias.data = torch.clamp(self.fc9.bias.data, min=dwn, max=up)
        x9 = F.leaky_relu((self.fc9(x8)), negative_slope=0.125) # x9: 9
        x9 = x9+x7[:,1:-1]

        self.fc10.weight.data *= torch.from_numpy(self.msk10).float()
        self.fc10.weight.data = torch.clamp(self.fc10.weight.data, min=dwn, max=up)
        self.fc10.bias.data = torch.clamp(self.fc10.bias.data, min=dwn, max=up)
        x10 = F.leaky_relu((self.fc10(x9)), negative_slope=0.125) # x10: 8
        x10 = x10+x8[:,1:-1]

        self.fc11.weight.data *= torch.from_numpy(self.msk11).float()
        self.fc11.weight.data = torch.clamp(self.fc11.weight.data, min=dwn, max=up)
        self.fc11.bias.data = torch.clamp(self.fc11.bias.data, min=dwn, max=up)
        x11 = F.leaky_relu((self.fc11(x10)), negative_slope=0.125) # x11: 7
        x11 = x11+x9[:,1:-1]

        self.fc12.weight.data *= torch.from_numpy(self.msk12).float()
        self.fc12.weight.data = torch.clamp(self.fc12.weight.data, min=dwn, max=up)
        self.fc12.bias.data = torch.clamp(self.fc12.bias.data, min=dwn, max=up)
        x12 = F.leaky_relu((self.fc12(x11)), negative_slope=0.125) # x12: 6
        x12 = x12+x10[:,1:-1]

        self.fc13.weight.data *= torch.from_numpy(self.msk13).float()
        self.fc13.weight.data = torch.clamp(self.fc13.weight.data, min=dwn, max=up)
        self.fc13.bias.data = torch.clamp(self.fc13.bias.data, min=dwn, max=up)
        x13 = F.leaky_relu((self.fc13(x12)), negative_slope=0.125) # x13: 5
        x13 = x13+x11[:,1:-1]

        self.fc14.weight.data *= torch.from_numpy(self.msk14).float()
        self.fc14.weight.data = torch.clamp(self.fc14.weight.data, min=dwn, max=up)
        self.fc14.bias.data = torch.clamp(self.fc14.bias.data, min=dwn, max=up)
        x14 = F.leaky_relu((self.fc14(x13)), negative_slope=0.125) # x14: 4
        x14 = x14+x12[:,1:-1]

        self.fc15.weight.data *= torch.from_numpy(self.msk15).float()
        self.fc15.weight.data = torch.clamp(self.fc15.weight.data, min=dwn, max=up)
        self.fc15.bias.data = torch.clamp(self.fc15.bias.data, min=dwn, max=up)
        x15 = F.leaky_relu((self.fc15(x14)), negative_slope=0.125) # x15: 3
        x15 = x15+x13[:,1:-1]

        self.fc16.weight.data *= torch.from_numpy(self.msk16).float()
        self.fc16.weight.data = torch.clamp(self.fc16.weight.data, min=dwn, max=up)
        self.fc16.bias.data = torch.clamp(self.fc16.bias.data, min=dwn, max=up)
        x16 = (self.fc16(x15)) # x16: 2
        x16 = x16+x14[:,1:-1]

        return x16

In [12]:
model = dianet20() # 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

min_loss = 100
for epoch in range(1,101):
    runloss = 0.0
    bs = 4
    count = 0
    for i in range(0, train_in.shape[0], bs):
        optimizer.zero_grad()
        outputs = model(train_in[i:i+bs], -1, 1)
        loss = criterion(outputs, train_target[i:i+bs])
        loss.backward()
        optimizer.step()
        runloss += loss.item()
        count += 1
    runloss = runloss / count

    model.eval()
    with torch.no_grad():
        test_out = model(test_in, -1, 1)
        test_loss = criterion(test_out, test_target)
        if test_loss < min_loss:
            min_loss = test_loss
            torch.save(model.state_dict(), 'jmeintdata/dianet-11.pth')
            print('saved at:', epoch, ' CEloss', test_loss.item())


saved at: 1  CEloss 0.6066116094589233
saved at: 3  CEloss 0.6066063642501831
saved at: 4  CEloss 0.6065918803215027
saved at: 5  CEloss 0.6065812706947327
saved at: 6  CEloss 0.6065694093704224
saved at: 7  CEloss 0.606560468673706
saved at: 8  CEloss 0.606548011302948
saved at: 9  CEloss 0.6065354943275452
saved at: 10  CEloss 0.606512188911438
saved at: 11  CEloss 0.6064926385879517
saved at: 12  CEloss 0.6064756512641907
saved at: 13  CEloss 0.6064572334289551
saved at: 14  CEloss 0.6064394116401672
saved at: 15  CEloss 0.6064161062240601
saved at: 16  CEloss 0.6063955426216125
saved at: 17  CEloss 0.6063697934150696
saved at: 18  CEloss 0.6063430309295654
saved at: 19  CEloss 0.6063051819801331
saved at: 20  CEloss 0.6062652468681335
saved at: 21  CEloss 0.6062349677085876
saved at: 22  CEloss 0.6062089204788208
saved at: 23  CEloss 0.6061959862709045


In [13]:
# test model
model = dianet20()

model.load_state_dict(torch.load('jmeintdata/dianet-11.pth'))
model.eval()
with torch.no_grad():
    outputs = model(test_in, -1, 1)
    _, predicted = torch.max(outputs.data, 1)
    print('Accuracy of test: %d %%' % (100 * torch.sum(test_target==predicted) / test_target.shape[0]))


Accuracy of test: 70 %


# clamp 0,1

In [14]:
model = dianet20() # 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

min_loss = 100
for epoch in range(1,101):
    runloss = 0.0
    bs = 4
    count = 0
    for i in range(0, train_in.shape[0], bs):
        optimizer.zero_grad()
        outputs = model(train_in[i:i+bs], 0, 1)
        loss = criterion(outputs, train_target[i:i+bs])
        loss.backward()
        optimizer.step()
        runloss += loss.item()
        count += 1
    runloss = runloss / count

    model.eval()
    with torch.no_grad():
        test_out = model(test_in, 0, 1)
        test_loss = criterion(test_out, test_target)
        if test_loss < min_loss:
            min_loss = test_loss
            torch.save(model.state_dict(), 'jmeintdata/dianet01.pth')
            print('saved at:', epoch, ' CEloss', test_loss.item())


saved at: 1  CEloss 0.6069663166999817
saved at: 2  CEloss 0.6068564057350159
saved at: 3  CEloss 0.6067605018615723
saved at: 4  CEloss 0.6066510081291199
saved at: 5  CEloss 0.6066004633903503
saved at: 6  CEloss 0.6065590381622314
saved at: 7  CEloss 0.6065300107002258
saved at: 8  CEloss 0.6065129041671753
saved at: 9  CEloss 0.6064874529838562
saved at: 10  CEloss 0.6064867973327637
saved at: 11  CEloss 0.6064832210540771
saved at: 14  CEloss 0.6064813733100891


In [15]:
# test model
model = dianet20()

model.load_state_dict(torch.load('jmeintdata/dianet01.pth'))
model.eval()
with torch.no_grad():
    outputs = model(test_in, 0, 1)
    _, predicted = torch.max(outputs.data, 1)
    print('Accuracy of test: %d %%' % (100 * torch.sum(test_target==predicted) / test_target.shape[0]))


Accuracy of test: 70 %
